In [ ]:
import os

In [ ]:
os.environ['KAGGLE_USERNAME']="raninaattef"
os.environ['KAGGLE_KEY']="a269bf45e22b6bed87e181047763d366"
!kaggle datasets download -d virenmohanlal/imdb-reviews-dataset #copy api

imdb-reviews-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls #bshwfha

imdb_reviews.csv  imdb-reviews-dataset.zip  sample_data


In [ ]:
!unzip imdb-reviews-dataset  #unzip b3d l /

Archive:  imdb-reviews-dataset.zip
replace imdb_reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: All
  inflating: imdb_reviews.csv        


In [ ]:
#used libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from collections import defaultdict
import math

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

In [ ]:
#read the data
reviews=pd.read_csv('/content/imdb_reviews.csv')


In [ ]:
#show the data
reviews
#we have 25000 reviews 

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative
...,...,...
24995,<START this is a racist movie but worthy of st...,positive
24996,<START bela lugosi plays a doctor who will do ...,negative
24997,<START in a far away galaxy is a planet called...,negative
24998,<START six degrees had me hooked i looked forw...,positive


In [ ]:
#some data structures
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Reviews    25000 non-null  object
 1   Sentiment  25000 non-null  object
dtypes: object(2)
memory usage: 390.8+ KB


In [ ]:
#check the nulls
reviews.isnull().sum()

Reviews      0
Sentiment    0
dtype: int64

In [ ]:
unique_sentiment=reviews['Sentiment'].unique()
unique_sentiment.sort()

In [ ]:
unique_sentiment   #check the inconsistent of sentiment

array(['negative', 'positive'], dtype=object)

In [ ]:
#a function to convert both lower and white spaces
def convert_lower_spaces(df,column):
    df[column] =df[column].str.lower()
    df[column] =df[column].str.strip()

In [ ]:
convert_lower_spaces(reviews,'Reviews')

In [ ]:

tokenizer = RegexpTokenizer(r'\w+')
STOP_WORDS = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
word_counts = defaultdict(lambda: [0, 0]) # returns [0, 0] by default if the key does not exist
sentiment = list(reviews['Sentiment'])

In [ ]:
#want to know the number of positive and negative in sentiment to start calculating probability
total_positive_reviews = 0
total_negative_reviews = 0

total_positive_words = 0
total_negative_words = 0

for index, review in enumerate(list(reviews['Reviews'])):
    if sentiment[index] == 'positive':
        total_positive_reviews += 1
    else:
        total_negative_reviews += 1
    
    for token in tokenizer.tokenize(review):
        token = token.lower()
        token = lemmatizer.lemmatize(token)
        if token not in STOP_WORDS:
            if sentiment[index] == 'positive':
                word_counts[token][1] += 1
                total_positive_words += 1
            else:
                word_counts[token][0] += 1
                total_negative_words += 1

In [ ]:
word_counts = sorted(word_counts.items(),  key=lambda x : x[1][0] + x[1][1], reverse=True)[:5500]

In [ ]:
word_counts = defaultdict(lambda: [0, 0], word_counts)

In [ ]:
#a function for calculating P(word|positive) and P(word|negative)

def calculate_word_probability(word, sentiment):
    if sentiment == 'positive':
        return math.log((word_counts[word][1] + 1) / (total_positive_words + 5500))
    else:
        return math.log((word_counts[word][0] + 1) / (total_negative_words + 5500))

In [ ]:
#a function for calculating each review  probability

def calculate_review_probability(review, sentiment):
    if sentiment == 'positive':
        probability = math.log(total_positive_reviews / len(reviews))
    else:
        probability = math.log(total_negative_reviews / len(reviews))
    
    for token in tokenizer.tokenize(review):
        token = token.lower()
        token = lemmatizer.lemmatize(token)
        if token not in STOP_WORDS:
            probability += calculate_word_probability(token, sentiment)
    return probability

In [ ]:
def predict(review):
    if calculate_review_probability(review, 'positive') > calculate_review_probability(review, 'negative'):
        return 'positive'
    else:
        return 'negative'

Testing 


In [ ]:
print(predict('This movie was great'))

positive


In [ ]:
#test on the entire dataset
correct  = 0
incorrect = 0
sentiments = list(reviews['Sentiment'])
for i, text in enumerate(list(reviews['Reviews'])):
    if predict(text) == sentiments[i]:
        correct += 1
    else:
        incorrect += 1

In [ ]:
Accuracy=(correct/(correct+incorrect))*100

In [ ]:
print('the accuracy of the Naive_Bayes Classifier is  {}'.format(Accuracy))

the accuracy of the Naive_Bayes Classifier is  85.63199999999999
